# H_22082024
# Analisis del volumen profile para encontrar zonas de baja cotización (FVG like) y como se comporta el precio en esas zonas.

En este Python Notebook analizaremos, con datos de mt5, como se comporta el mercado en las zonas previamente definidas.

## Conceptos clave del perfil de mercado:

1. **Oportunidad de precio en el tiempo (TPO):**
    
    representa los niveles de precio negociados durante intervalos de tiempo específicos. En un gráfico de perfil de mercado, las TPO indican la frecuencia con la que se negoció cada nivel de precio durante una sesión.
    
2. **Área de valor:**
    
    rango de precios en el que se produjo el 70 % de la actividad comercial durante una sesión. Esta área representa el valor justo percibido del mercado.
    
3. **Punto de control (POC):**
    
    el nivel de precio con el mayor volumen durante la sesión. El POC suele ser un nivel clave de soporte o resistencia.
    
4. **Balance inicial (IB):**
    
    rango de precios negociados durante la primera hora de la sesión de negociación. Proporciona una visión anticipada de la estructura del mercado del día.

In [115]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [131]:
# Variables

ticker = "GC_V"

## 1. Definición de zonas de baja cotización para posterior analisis.
El primer paso a dar es definir como encontraremos las **zonas de baja cotización** en una sesion de mercado de forma objetiva.

A continuación, definiremos que es una zona de baja cotización para nuestro estudio:

1. Analisis basado en el *volumen profile*, el cual analiza el numero de cotizaciónes por precio que tiene lugar en una sesión de trading predefinida (en nuestro estudio, una sesión será un dia 1D).
2. La media y la StdDev del volumen profile de la sesión nos ayudarán a analizar correctamente las zonas.

Por tanto, una **zona de baja cotización** consiste en:
- Los niveles de precio que se encuentran N*StdDev(mean(volume_profile)) por debajo.

In [132]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# request connection status and parameters
print(mt5.terminal_info())
# get data on MetaTrader 5 version
print(mt5.version())

# set time zone to UTC
timezone = pytz.timezone("Etc/UTC")
# create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
utc_from = datetime(2024, 6, 10, tzinfo=timezone)
utc_to = datetime(2024, 6, 30, tzinfo=timezone)

# request AUDUSD ticks within 11.01.2020 - 11.01.2020
ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, utc_from, utc_to)
daily_ohlcv = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, utc_from, utc_to)

mt5.shutdown()

TerminalInfo(community_account=True, community_connection=True, connected=True, dlls_allowed=True, trade_allowed=False, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=True, build=4410, maxbars=100000, codepage=1252, ping_last=40131, community_balance=0.0, retransmission=4.310665180230398, company='MetaQuotes Software Corp.', name='MetaTrader 5', language='Spanish', path='C:\\Program Files\\MetaTrader 5', data_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\D0E8209F77C8CF37AD8BF550E51FF075', commondata_path='C:\\Users\\iamfr\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')
(500, 4410, '21 Jun 2024')


True

In [133]:
# convert time in seconds into the datetime format
df = pd.DataFrame(ohlcv)
df['time']=pd.to_datetime(df['time'], unit='s')

# adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
daily_df = pd.DataFrame(daily_ohlcv)
daily_df['time']=pd.to_datetime(df['time'], unit='s')

df = df.set_index('time')
del df['real_volume']
del df['spread']

daily_df = daily_df.set_index('time')
del daily_df['real_volume']
del daily_df['spread']

In [134]:
def create_market_profile(data):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = profile.loc[profile['tick_volume'].idxmax(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

Este codigo no es necesario para la ejecución normal del analisis. Este nos ayuda a visualizar el perfil de volumen de una sesión, sin los niveles que estan por debajo de el threshold.

```
no_fair_value_prices = profile.copy() # En realidad son los precios que superan el threshold, pero mas tarde buscaremos los huecos.

no_fair_value_prices['tick_volume'] = no_fair_value_prices['tick_volume'][no_fair_value_prices['tick_volume'] > threshold]

plt.figure(figsize=(10, 6))
plt.barh(no_fair_value_prices['close'], no_fair_value_prices['tick_volume'], color='blue', edgecolor='black')
plt.xlabel('Volume')
plt.ylabel('Price')
plt.title(f'Market Profile')
plt.legend()
plt.show()
```

In [135]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [136]:
# iteramos cada dia, y cada zona
def get_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * 0.5

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)
        
        output = {
            "date": index1.strftime('%Y-%m-%d'),
            "zone": no_fair_value_zone,
        }
        out.append(output)

    return out

## Analisis de datos

A partir de aqui, ya disponemos de las herramientas de manipulación de los datos, como para poder empezar a analizarlos.

Disponemos de una lista "ZONES_BY_DAY", la qual nos permitirá referenciar las zonas de baja cotización por dia, y sus niveles de precio. Asi podemos hacer un analisis más comodo.

### 1. Next session close zone

Analizaremos si en la siguiente sesión, el precio cierra el "vacio" creado por la zona de baja cotización del dia anterior.

In [ ]:
def num1_next_session_close_zone():
    
    return False